# Design of an A/B Test : Free Trial Screener
by Julien Delbove




## Experiment Design
### Metric Choice

In the experiment, I will use the following invariant metric:
* Number of cookies
* Number of clicks
* Click-through-probability  

As evaluation metrics, I choose:
* Gross conversion
* Retention 
* Net conversion


I made these decisions for the following reasons:
* Number of cookies (Number of unique users to visit the course overview page): Since the unit of diversion is the cookies, they must be evenly distributed between the control and the experiment groups. Therefore, I will use this metric as an invariant.
* Number of user-ids (Number of users who enroll in the free trial): This metric is not an invariant since the experiment is expected to provide warning to the user and reduce the number of enrollment from people who can't commit enough hours. 
This metric could potentially be used as an evaluation metrics because it is expected to see a difference between the experiment and the control group. However, since it is an absolute value, it is difficult to have an a-priori idea of the distribution that can be expected for this value. The use of this metric would make the experiment more difficult to analyse. Since other metrics are possible, I would not choose this one.
* Number of clicks (Number of unique cookies to click the "Start free trial" button):Since the experiment happen after the click on the "Start free trial", this metric can be used as an invariant.
* Click-through-probility (Number of unique cookies to click the "Start free trial" button divided by the number of unique cookies to view the course overview page): This metric is based on 2 metrics that are invariants. Therfore it is an invariant.
* Gross conversion (Number of user-ids to complete checkout and enroll in the free trial divided by the number of unique cookies to click the "Start free trial" button): This metric is an evaluation metric because the ratio difference between the experiment and the control group will give the impact of the warming on the people who decide to enroll in the free trial.
* Retention (Number of user-ids to remain enroll past the 14-day boundary divided by the number of user-ids to complete checkout): This metric could also be considered as an evaluation metrics since it measures the impact of the experiment on the ratio of users remaining enrolled pas the 14-day boundary.  
* Net conversion (Number of user-ids to remain enroll past the 14-day boundary divided by the number of unique cookies to click the "Start free trial" button). I choose this metric as an evaluation metric since it give the impact of the experiment on the people who remain enroll past the 14-day boundary.


The results I expect for my chosen evaluation metrics are the following:
* Gross conversion: if the experiment works as expected, this metric should be lower for the experiment group than for the control group. Indeed, once the pop-up as been displayed to the visitor, I expect, he will enroll only if he evaluates that he could commit the requested amount of time. Therefore, less visitors should enroll in the experiment group.
* Retention: If only warned people enroll in the 14-day trial, I expect that more of them will remain after the 14-day boundary. Therefore this metric should be higher in the experiment group than in the control group.
* Net conversion: I expect the net conversion to remain the same or to increase between the experiment and the control group. It measures how many people remains enroll after the 14-day boundary over the number of unique cookies to click the "Start free trial". In the experiment group, enrolled people have been warned, therefore there will be less enrollment but it means also that more of them should remain enrolled after the 14-day boundary. In the control group, people are not warned, therefore there will be more enrollment but it means that more of them will leave the course in-between. At the end, the final number of people remaining engaged over the number of click on the "Start free trial" should be the same between the experiment and the control group. The advantage for Udacity is that udacity coaches will be less solicitated by people that are in the free trial period and that might leave early because they can't commit to enough learning hours. The coachs will then have more time to help the people who commit to enough hours during the 14-days boundary and it might result that a higher number remain enrolled.


### Measuring Standard Deviation

In [20]:
import math

# Analytical standard deviation evaluation

# Inputs
unique_cookies_per_day=5000
click_through_probability_start_free_trial=0.08
probability_of_enrolling_given_click=0.20625
probability_of_payment_given_enroll=0.53
probability_of_payment_given_click=0.1093125

# Number of click on "Start free trial"
Nclick=unique_cookies_per_day*click_through_probability_start_free_trial

# Number of enrollment
Nenroll=Nclick*probability_of_enrolling_given_click

# Evaluation of standard deviation of binomial distribution B(p,N)
standard_dev_gross_conversion=math.sqrt(
    probability_of_enrolling_given_click*(1.-probability_of_enrolling_given_click)/Nclick)

standard_dev_retention=math.sqrt(
    probability_of_payment_given_enroll*(1.-probability_of_payment_given_enroll)/(Nenroll))

standard_dev_net_conversion=math.sqrt(
    probability_of_payment_given_click*(1.-probability_of_payment_given_click)/Nclick)



print "Standard deviation of Gross conversion for a sample size of 5000 cookies: %.4f"%standard_dev_gross_conversion
print "Standard deviation of Retention for a sample size of 5000 cookies: %.4f"%standard_dev_retention
print "Standard deviation of Net conversion for a sample size of 5000 cookies: %.4f"%standard_dev_net_conversion


Standard deviation of Gross conversion for a sample size of 5000 cookies: 0.0202
Standard deviation of Retention for a sample size of 5000 cookies: 0.0549
Standard deviation of Net conversion for a sample size of 5000 cookies: 0.0156


The three evaluation metrics are probabilities. Their distributions are likely to follow Binomial distributions. The standard deviation for my evaluation metrics for a sample size of 5000 cookies are:
* 0.0202 for the Gross conversion
* 0.0549 for the Retention
* 0.0156 for the Net conversion  

The unit of diversion is a cooky. The units of analysis for Gross Conversion and Net Conversion are also cookies (the denominator of the metrics). Therefore the variability computed analitically is likely to a be very close to the empirically computed variability. The units of analysis for the Retention is the number of user-ids. As the unit of diversion and the unit of analysis are not the same, the empirical variability may be much higher than the analytical estimate.

### Sizing
#### Number of Samples vs. Power  


In [21]:
# Number of sample for one group for each metric:
# Gross Conversion
N1=25835
# Retention
N2=39115
# Net Conversion
N3=27413

# Number of pageviews necessary for each evaluation metrics
Np1=N1/click_through_probability_start_free_trial
Np2=N2/(click_through_probability_start_free_trial*probability_of_enrolling_given_click)
Np3=N3/click_through_probability_start_free_trial

print "The number of pageviews necessary for the Gross conversion (one group) is %i"%Np1
print "The number of pageviews necessary for the Retention (one group) is %i"%Np2
print "The number of pageviews necessary for the Net conversion (one group) is %i"%Np3

# Number of sample for the experiment for both groups:
N=2*max(Np1,Np2,Np3)

print "\n"
print "The number of pageviews necessary for the experiment is %i"%N

The number of pageviews necessary for the Gross conversion (one group) is 322937
The number of pageviews necessary for the Retention (one group) is 2370606
The number of pageviews necessary for the Net conversion (one group) is 342662


The number of pageviews necessary for the experiment is 4741212


I will not use the Bonferroni correction during my analysis phase. In order to compute the number of clicks on the "Start free trial" button that will be needed, I used the calculator available at the address: [Calculator](http://www.evanmiller.org/ab-testing/sample-size.html). 
I get the following results:
* 25835 samples necessary for the Gross conversion to identify a 1% variation with $\alpha=0.05$ and $\beta=0.2$.
* 39115 samples necessary for the Gross conversion to identify a 1% variation with $\alpha=0.05$ and $\beta=0.2$.
* 27413 samples necessary for the Net conversion to identify a 0.75% variation with $\alpha=0.05$ and $\beta=0.2$.

In order to be able to get a proper evaluation of the evolution of both metrics, I will consider the maximum value of the figures obtained with the calculator. This number of sample is for one group. For the Gross conversion and the Net convertion, only 8% of the cookies will lead to a click on the "strat free trial" button. For the Retention, as the unit of analysis is the number of user-ids, an estimation of the percentage of cookies leading to an enrollment is: $0.08*0,20625$. It gives the following number of cookies for one group for each of the metrics:
* 322937 for the Gross Conversion  
* 2370606 for the Retention
* 342662 for the Net Conversion  
Since there are 2 groups and the cookies will be balanced between both group, the number of samples necessary for both groups for all the metrics is equal to $N=2*max(322937,2370606,342662). It gives 4741212 cookies.

#### Duration vs. Exposure


In [22]:
# fraction of traffic to be diverted for a one month experiment
f=1.

# Number of days with a fraction of f for all 3 metrics
N=2*max(Np1,Np2,Np3)
Nd_3metrics=math.ceil(N/(40000*f))

print "The number of days for a fraction of divertion equal to %.2f is equal to %i"%(f,Nd_3metrics)

# Number of days with a fraction of f for Gross conversion and Net conversion
N=2*max(Np1,Np3)
print "The number of cookies necessary for the experiment is %i"%N
Nd=math.ceil(N/(40000*f))

print "The number of days for a fraction of divertion equal to %.2f is equal to %i"%(f,Nd)

The number of days for a fraction of divertion equal to 1.00 is equal to 119
The number of cookies necessary for the experiment is 685325
The number of days for a fraction of divertion equal to 1.00 is equal to 18


For this experiment, I would divert the complete traffic. It gives a number of 119 days for the experiment. As it not acceptable for Udacity to make an experiment for so long, the Retention, which is the evaluation metric leading to such along time, will be excluded from the evaluation metrics. With only Gross conversion and Net Conversion as evaluation metrics, the experiment requires 685325 cookies which will be obtained in 18 days.

It is acceptable to divert the complete traffic for the following reasons:
* The risk for the participant is very minor. It consists only of a warning to the user which can't lead to any harm in any of the following categories: physical, psychological and emotional, social, and economic.
* In term of benefit, the participant is warned and it can prevent him to lose time and being frustrated.
* In term of alternative, the participant is asked to enter a number of hours. However the user has the possibility to register if he say that he will commit to enough hours.
* The data sensitivity requested by this experiment is very low. It asks a number of hours that the users can commit. Even if the user can be identified based on the cooky, a number of hour committed to learning is not sensitive.

The only risk would be that the warning results in a significant decrease of net conversion and then loss of money for udacity because of the experiment. A decrease of the Net conversion of 0.75% would result in a loss of money of at least 40000\$ ($0.0075*0.08*685325/2*200=41119$ with 200\$ being the fee for one month subscription). At this point, it is up to the decision makers to decide wether to do the experiment or not.

## Experiment Analysis
### Sanity Checks


In [23]:
import pandas as pd

control=pd.read_csv("Final Project Results - Control.csv", sep=',')
exp=pd.read_csv("Final Project Results - Experiment.csv", sep=',')

print control.head(5)
print "================================================="
print exp.head(5)


Npageview_c=control["Pageviews"].sum()
Npageview_e=exp["Pageviews"].sum()

Nclicks_c=control["Clicks"].sum()
Nclicks_e=exp["Clicks"].sum()

# Sanity check for Number of cookies
p_theory1=0.5
p_observed1=float(Npageview_c)/(Npageview_c+Npageview_e)
m1=1.96

SD1=math.sqrt(p_theory1*(1.-p_theory1)/(Npageview_c+Npageview_e))
LB1=p_theory1-m1*SD1
UB1=p_theory1+m1*SD1
print "\n"
print "Lower Bound for Number of cookies is %.4f"%LB1
print "Upper Bound for Number of cookies is %.4f"%UB1
print "Observed probability is %.4f"%p_observed1

# Sanity check for Number of clicks
p_theory2=0.5
p_observed2=float(Nclicks_c)/(Nclicks_c+Nclicks_e)
m2=1.96

SD2=math.sqrt(p_theory2*(1.-p_theory2)/(Nclicks_c+Nclicks_e))
LB2=p_theory2-m2*SD2
UB2=p_theory2+m2*SD2
print "\n"
print "Lower Bound for Number of clicks is %.4f"%LB2
print "Upper Bound for Number of clicks is %.4f"%UB2
print "Observed probability is %.4f"%p_observed2

# Sanity check for click-through-probability
p_theory3=0.
p_observed3=float(Nclicks_c)/Npageview_c-float(Nclicks_e)/Npageview_e
m3=1.96

p_pooled=float((Nclicks_c+Nclicks_e))/(Npageview_c+Npageview_e)
SD3=math.sqrt(p_pooled*(1.-p_pooled)*(1./Npageview_c+1./Npageview_e))
LB3=p_theory3-m3*SD3
UB3=p_theory3+m3*SD3
print "\n"
print "Lower Bound for difference of click-through-probability is %.4f"%LB3
print "Upper Bound for difference of click-through-probability is %.4f"%UB3
print "Observed difference of click-through-probability is %.4f"%p_observed3



          Date  Pageviews  Clicks  Enrollments  Payments
0  Sat, Oct 11       7723     687        134.0      70.0
1  Sun, Oct 12       9102     779        147.0      70.0
2  Mon, Oct 13      10511     909        167.0      95.0
3  Tue, Oct 14       9871     836        156.0     105.0
4  Wed, Oct 15      10014     837        163.0      64.0
          Date  Pageviews  Clicks  Enrollments  Payments
0  Sat, Oct 11       7716     686        105.0      34.0
1  Sun, Oct 12       9288     785        116.0      91.0
2  Mon, Oct 13      10480     884        145.0      79.0
3  Tue, Oct 14       9867     827        138.0      92.0
4  Wed, Oct 15       9793     832        140.0      94.0


Lower Bound for Number of cookies is 0.4988
Upper Bound for Number of cookies is 0.5012
Observed probability is 0.5006


Lower Bound for Number of clicks is 0.4959
Upper Bound for Number of clicks is 0.5041
Observed probability is 0.5005


Lower Bound for difference of click-through-probability is -0.0013
Upper B

The sanity checks for the invariant metrics give:
* Number of cookies
    * Lower Bound: 0.4988
    * Upper Bound: 0.5012
    * Observed Probability: 0.5006
    * Since the Observed Probability is between the lower and the upper bound of the 95% confidence interval, the sanity check pass
* Number of clicks:
    * Lower Bound: 0.4959
    * Upper Bound: 0.5041
    * Observed Probability: 0.5005
    * For the same reason as for the Number of cookies, the sanity check pass for the Number of clicks
* Click-through-probability
    * Lower Bound: -0.0013
    * Upper Bound: 0.0013
    * Observed Probability: -0.0001
    * The observed difference of click-through probability is between the lower and upper bound of the 95% confidence interval, so the sanity check pass.

### Result Analysis
#### Effect Size Tests

In [24]:
# Remove lines with NaN for evaluation Metrics
control_sub=control.dropna()
exp_sub=exp.dropna()

Nclicks_c=control_sub["Clicks"].sum()
Nclicks_e=exp_sub["Clicks"].sum()

Nenrollments_c=control_sub["Enrollments"].sum()
Nenrollments_e=exp_sub["Enrollments"].sum()

Npayments_c=control_sub["Payments"].sum()
Npayments_e=exp_sub["Payments"].sum()

# Evaluation of 95% confidence interval for Gross conversion
p_observed1=float(Nenrollments_e)/Nclicks_e-float(Nenrollments_c)/Nclicks_c
m1=1.96

p_pooled1=float((Nenrollments_c+Nenrollments_e))/(Nclicks_c+Nclicks_e)
SD1=math.sqrt(p_pooled1*(1.-p_pooled1)*(1./Nclicks_c+1./Nclicks_e))
LB1=p_observed1-m1*SD1
UB1=p_observed1+m1*SD1
print "\n"
print "Lower Bound for difference of Gross Conversion difference is %.4f"%LB1
print "Upper Bound for difference of Gross Conversion difference is %.4f"%UB1

# Evaluation of 95% confidence interval for Net conversion
p_observed2=float(Npayments_e)/Nclicks_e-float(Npayments_c)/Nclicks_c
m2=1.96

p_pooled2=float((Npayments_c+Npayments_e))/(Nclicks_c+Nclicks_e)
SD2=math.sqrt(p_pooled2*(1.-p_pooled2)*(1./Nclicks_c+1./Nclicks_e))
LB2=p_observed2-m2*SD2
UB2=p_observed2+m2*SD2
print "\n"
print "Lower Bound for difference of Net Conversion difference is %.4f"%LB2
print "Upper Bound for difference of Net Conversion difference is %.4f"%UB2





Lower Bound for difference of Gross Conversion difference is -0.0291
Upper Bound for difference of Gross Conversion difference is -0.0120


Lower Bound for difference of Net Conversion difference is -0.0116
Upper Bound for difference of Net Conversion difference is 0.0019


The results for the evaluation metrics are:
* Gross Conversion 
    * The Confidence Interval is [-0.0291,-0.0120]
    * The difference in the Gross Conversion probability is statistically significant since zero is not in the confidence interval
    * The result is also practically significant since $d_{min}$ is not in the interval
* Net conversion
    * The Confidence Interval is [-0.0116,0.0019]
    * The difference in the Net Conversion probability is not statistically significant since zero is included in the confidence interval.
    * The result is not practically significant since $d_{min}$ is included in the interval

#### Sign Tests


In [25]:
import numpy as np

control_sub.is_copy = False
exp_sub.is_copy = False

# Computation of Gross conversion probability per day
control_sub["Gross_Conversion"]=control_sub["Enrollments"]/control_sub["Clicks"]
#control_sub["Gross_Conversion"]=control_sub[:,"Enrollments"]/control_sub[:,"Clicks"]

exp_sub["Gross_Conversion"]=exp_sub["Enrollments"]/exp_sub["Clicks"]

# Computation of Net conversion probability per day
control_sub["Net_Conversion"]=control_sub["Payments"]/control_sub["Clicks"]
exp_sub["Net_Conversion"]=exp_sub["Payments"]/exp_sub["Clicks"]

# Computation sign tables
sign_diff_Gross_Conversion=np.sign(exp_sub["Gross_Conversion"]-control_sub["Gross_Conversion"])
sign_diff_Net_Conversion=np.sign(exp_sub["Net_Conversion"]-control_sub["Net_Conversion"])

print "Number of success for Gross Conversion: %i"%((sign_diff_Gross_Conversion==-1.).sum())
print "Number of success for Net Conversion: %i"%((sign_diff_Net_Conversion==-1.).sum())
print "Number of trials: %i"%(len(sign_diff_Gross_Conversion))


Number of success for Gross Conversion: 19
Number of success for Net Conversion: 13
Number of trials: 23


The sign tests for the evaluation metrics give the following results:
* Gross Conversion 
    * 19 successes (number of days where the probability of Enrollment given clicks for experiment group is below the one of the control group)
    * 23 days are recorded
    * with the p-value calculator proposed at the address: [p-value Calculator](http://graphpad.com/quickcalcs/binomial1.cfm), it gives the value 0.0026
    * As this value is below the $\alpha$ value, the result of the sign test is statistically significant.
* Net Conversion
    * 13 successes
    * 23 trials
    * p-value is equal to 0.6776
    * the difference is not statistically significant


#### Summary

The Bonferroni correction is not required since both evaluation metrics need to match the expectations. As the expectation for both metrics need to be matched to launch the change, there is no need to use the Bonferroni correction in order to reduce the risk of having one of the metric matching the expectation by chance. Even in this case the experiment wouldn't be launched.  
The effect size hypothesis tests and the sign tests are coherents and give the same conclusions.


### Recommendation

The experiment gives the following results:
* less people enroll for the free trial: the Gross conversion is statistically and practically lower which means that the warming on the mandatory commitment has the expected impact.
* the impact on the number of people remaining enrolled after the 14-days is neither practically or statistically significant. The practical level of significance (-0.0075) is included in the confidence interval [-0.0116,0.0019].   

There is the a risk that the net conversion is significantly decreased because of the experiment. As a conclusion, I would recommend to not apply the change to the udacity website.


## Follow-Up Experiment

The results obtained in the experiment have a real impact on the people who enroll in the course. However, since there is a high risk of having a decrease of the Net conversion, it also seems that the warning is a bit to hard: it prevents people that evaluate less than 5h per week to enroll whereas depending on their background, less than 5h might enough for them. 

As a follow up experiment I would slightly modify the warning by making the number of hour required depending on the background of the people. In addition to the number of hours committed to learning, I would add an additional question like: How many years of experience have you got in a related field (0, 1-3, 3+). For 0 years of experience in a related field, the minimum number of hours could become 7h, for 1-3 years of experience, 5 years could remains the limit. For people, with 3+ years of experience in related field (software development, mathematics, engineering,...), the minimum number of hour could be 3h.

As for the original experience, my hypothesis is that the gross conversion will decrease since clear expectations are set depending on the background of the people (maybe a bit less than the original experiment). But since the warning is better adapted to the people background, I expect that the Net conversion will remain the same: the warning should not prevent anybody with an adequate number of hours committed to learning with regards to their experience to enroll.

I would use the same metrics and the same units of diversion since the experiment is the same expect for the screener.
